## Preparación de texto para Machine Learning

Los datos de texto requieren una preparación especial antes de poder ser usados en modelos predictivos.

El texto debe analizarse para eliminar **STOPWORDS**. Luego las palabras deben transformarse a valores numéricos para poder ser usado por lo modelos de predicción, este proceso se llama **`Feature Extraction`** o **`Vectorization`** (vectorizacion).

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizacion
from sklearn.preprocessing import MinMaxScaler

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Activation
from tensorflow.keras.models import Sequential

# Validacion
from sklearn.model_selection import StratifiedKFold

In [24]:
df = pd.read_csv("../Data/movie_data.csv")

df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
df.shape

(50000, 2)

In [5]:
df["sentiment"].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [6]:
print(df.loc[0, "review"])

In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70's, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich family 

### Bag-of-Words

Para poder trabajar con texto debemos transformar el texto en números, un modelo simple y efectivo es el modelo **Bag-of-Words**.

El modelo transforma el texto en una matriz, donde las palabras son columnas y las filas son las ocurrencias o las veces que aparece cada palabra en todo el texto.

En Sklearn tenemos la función **`CountVectorizer()`**.

_**Documentación:** https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html_

In [12]:
from sklearn.feature_extraction.text import CountVectorizer


# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer()

frases = np.array(["The sun is shining",
                   "The weather is sweet", 
                   "The sun is shining, the weather is sweet, and one and one is two and three"])

# Entrenamos el modelo y transformamos los datos.
bag = count_vectorizer.fit_transform(frases)

bag

# bag es una matriz sparse, por lo que Python decide mostrarla de la siguiente manera

<3x10 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [13]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
       [3, 3, 2, 1, 1, 1, 2, 1, 1, 1]], dtype=int64)

In [14]:
# .vocaboulary_ retorna cada palabra con su indice correspondiente en un diccionario.

print(count_vectorizer.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 9, 'sweet': 5, 'and': 0, 'one': 2, 'two': 8, 'three': 7}


In [15]:
vocabulario_ordenado = sorted(count_vectorizer.vocabulary_, key = lambda x : count_vectorizer.vocabulary_[x]) 
vocabulario_ordenado

['and',
 'is',
 'one',
 'shining',
 'sun',
 'sweet',
 'the',
 'three',
 'two',
 'weather']

In [16]:
pd.DataFrame(data = bag.toarray(), columns = vocabulario_ordenado)

,and,is,one,shining,sun,sweet,the,three,two,weather
0,0,1,0,1,1,0,1,0,0,0
1,0,1,0,0,0,1,1,0,0,1
2,3,3,2,1,1,1,2,1,1,1


### Term Frequency-Inverse Document Frequency (TF-IDF)

La siguiente técnica se puede traducir como **frecuencia de término-frecuencia inversa de documento**, se utiliza para disminuir el peso de las palabras que aparecen muchas veces en multiples documentos, la ecuación es la siguiente: 

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

* El producto de la frecuencia de término $(tf(t,d))$ y la frecuencia inversa de documento $(idf(t,d))$. Donde ***t*** es el número de terminos y ***d*** el número de documento.

* ***idf(t,d)*** se calcula con la siguiente ecuación:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

* Donde $n_d$ es el número total de documentos 
* ***df (d,t)*** es el número de documentos ***d*** que contienen el término ***t*** 
* El logaritmo se utiliza para evitar que las bajas frecuencias de documentos no adquieran demasiado peso

#### Ejemplo:
$$\text{idf}("is", d3) = log \frac{3}{1+3} = -0.287$$


$$\text{tf-idf}("is",d3)= 3 \times (-0.287) = -0.863$$

**Usos**

- `Indexación de texto`: Tf-idf se puede utilizar para indexar texto. Esto ayuda a los sistemas de búsqueda a encontrar texto relevante para una consulta.
- `Filtrado de ruido`: Tf-idf se puede utilizar para filtrar ruido del texto. Esto ayuda a mejorar la precisión de los sistemas de PLN.
- `Clasificación de texto`: Tf-idf se puede utilizar para clasificar texto. Esto ayuda a los sistemas de PLN a determinar el tema de un texto.
- `Relevancia`: Tf-idf se puede utilizar para medir la relevancia entre dos documentos. Esto ayuda a los sistemas de PLN a determinar qué documentos son relevantes para una consulta.

**Ventajas**

- Es simple de calcular.
- Es robusta a los cambios en el tamaño del documento.
- Es efectiva para una variedad de tareas de procesamiento del lenguaje natural.

**Desventajas** 

- Puede ser sensible a la frecuencia de las palabras.
- Puede ser sesgada hacia las palabras que aparecen con frecuencia en un dominio específico.

En Sklearn tenemos la función **`TfidfTransformer()`**.

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Cambio la precisión de python a 2 decimales
np.set_printoptions(precision = 2)


# Entrenamos el Tfidf y transformamos la variable bag
bag_tfidf = tfidf.fit_transform(bag).toarray()

bag_tfidf

array([[0.  , 0.43, 0.  , 0.56, 0.56, 0.  , 0.43, 0.  , 0.  , 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.  , 0.56, 0.43, 0.  , 0.  , 0.56],
       [0.64, 0.38, 0.43, 0.16, 0.16, 0.16, 0.25, 0.21, 0.21, 0.16]])

In [18]:
pd.DataFrame(data = bag_tfidf, columns = vocabulario_ordenado)

,and,is,one,shining,sun,sweet,the,three,two,weather
0,0.000000,0.433708,0.000000,0.558478,0.558478,0.000000,0.433708,0.000000,0.000000,0.000000
1,0.000000,0.433708,0.000000,0.000000,0.000000,0.558478,0.433708,0.000000,0.000000,0.558478
2,0.641818,0.379068,0.427879,0.162706,0.162706,0.162706,0.252712,0.213939,0.213939,0.162706


**Vamos a aplicar ambas técnicas a `movie_data.csv`.**

In [25]:
df = df.iloc[15_000: -15_000, :]

In [26]:
df

,review,sentiment
15000,"Take a few dark and stormy nights, fog coming ...",0
15001,"Hitokiri (which translates roughly as ""assassi...",1
15002,I saw Forgiving the Franklins at a Paramount s...,1
15003,"In a film as successful as this, it is difficu...",1
15004,"Starts really well, nice intro and build up fo...",0
...,...,...
34995,"Okay, last night, August 18th, 2004, I had the...",0
34996,Terrible psychological thriller that is almost...,0
34997,CAROL'S JOURNEY is a pleasure to watch for so ...,1
34998,"When, oh, when will someone like Anchor Bay or...",1


In [27]:
# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer()

# Entrenamos el modelo y transformamos los datos.
bag = count_vectorizer.fit_transform(df["review"])

bag

<20000x69431 sparse matrix of type '<class 'numpy.int64'>'
	with 2724428 stored elements in Compressed Sparse Row format>

In [28]:
# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Entrenamos el Tfidf y transformamos la variable bag
bag_tfidf = tfidf.fit_transform(bag)

bag_tfidf

<20000x69431 sparse matrix of type '<class 'numpy.float64'>'
	with 2724428 stored elements in Compressed Sparse Row format>

In [ ]:
len(bag_tfidf.toarray()[0])

In [ ]:
# X = bag_tfidf
# y = df["sentiment"].values

# X.shape, y.shape

In [ ]:
# del bag_tfidf, df

In [32]:
X_train, X_test, y_train, y_test = train_test_split(bag_tfidf, df["sentiment"].values,
                                                    test_size = 0.3, random_state = 42, stratify = df["sentiment"].values)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

X_train: (14000, 69431), y_train: (14000,)
X_test: (6000, 69431),  y_test: (6000,)


In [33]:
%%time

# GaussianNB()

model = GaussianNB()

# Entrenamiento
model.fit(X_train.toarray(), y_train)

# Predicciones
yhat = model.predict(X_test.toarray())

# Métricas
print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
print("Accuracy:"     , accuracy_score(y_test, yhat))
print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
print("ROC AUC:"      , roc_auc_score(y_test, yhat))

Jaccard Index: 0.47963560698781166
Accuracy: 0.6491666666666667
Precisión: 0.6512360415500225
Sensibilidad: 0.649224482557816
F1-score: 0.6480163744008858
ROC AUC: 0.6492244825578158
CPU times: total: 26.5 s
Wall time: 1min 23s


In [34]:
vocabulario_ordenado = sorted(count_vectorizer.vocabulary_, key = lambda x : count_vectorizer.vocabulary_[x]) 
vocabulario_ordenado

['00',
 '000',
 '0000000000001',
 '000dm',
 '001',
 '007',
 '0079',
 '0080',
 '0083',
 '009',
 '0093638',
 '00am',
 '00o',
 '00pm',
 '00s',
 '01',
 '02',
 '020410',
 '0230',
 '029',
 '03',
 '04',
 '041',
 '044',
 '05',
 '06',
 '07',
 '07b',
 '08',
 '087',
 '089',
 '09',
 '0f',
 '0r',
 '10',
 '100',
 '1000',
 '10000',
 '1000000',
 '10000000000000',
 '1000lb',
 '1000s',
 '1001',
 '1004',
 '100b',
 '100ibs',
 '100k',
 '100m',
 '100miles',
 '100mins',
 '100mph',
 '100s',
 '100th',
 '100x',
 '101',
 '101st',
 '102',
 '103',
 '104',
 '1040s',
 '105',
 '106',
 '107',
 '108',
 '109',
 '10k',
 '10min',
 '10minutes',
 '10pm',
 '10s',
 '10th',
 '10x',
 '11',
 '110',
 '110min',
 '110mph',
 '111',
 '112',
 '113',
 '1138',
 '113min',
 '113minutes',
 '114',
 '1146',
 '115',
 '116',
 '116minutes',
 '117',
 '118',
 '119',
 '11am',
 '11th',
 '11ths',
 '12',
 '120',
 '1200',
 '1200f',
 '1201',
 '1202',
 '120kmph',
 '121',
 '122',
 '123',
 '123k',
 '124',
 '1242',
 '125',
 '125m',
 '127',
 '128',
 '129',


In [35]:
for i in vocabulario_ordenado:
    print(i)

00
000
0000000000001
000dm
001
007
0079
0080
0083
009
0093638
00am
00o
00pm
00s
01
02
020410
0230
029
03
04
041
044
05
06
07
07b
08
087
089
09
0f
0r
10
100
1000
10000
1000000
10000000000000
1000lb
1000s
1001
1004
100b
100ibs
100k
100m
100miles
100mins
100mph
100s
100th
100x
101
101st
102
103
104
1040s
105
106
107
108
109
10k
10min
10minutes
10pm
10s
10th
10x
11
110
110min
110mph
111
112
113
1138
113min
113minutes
114
1146
115
116
116minutes
117
118
119
11am
11th
11ths
12
120
1200
1200f
1201
1202
120kmph
121
122
123
123k
124
1242
125
125m
127
128
129
12a
12nd
12p
12th
12x
13
130
131
132
134
135
135m
136
137
138
13848
13itching
13th
14
140
1408
140hp
1415
142
1453
1454
146
1492
14a
14s
14th
15
150
1500
1500s
150_worst_cases_of_nepotism
152
155
156
1561
157
157th
158
1596
1598
15apr08
15ft
15k
15mins
15pm
15th
16
160
1600
1600s
161
1611
162
163
1647
165
166
1660s
168
1683
1692
16ieme
16mm
16s
16th
16ème
17
170
1700
1701
1709s
170x
1720
175
1759
176th
1775
1776
1780s
1789
1790
1794
1798
17

In [ ]:
################################################################################################################################